In [1]:
from dash import Dash, dcc, html, Input, Output, State, no_update
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv'

airline_data = pd.read_csv(URL)
airdata = airline_data.sample(n = 500, random_state = 42)

In [3]:
app = Dash(__name__)

# Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# list of years:
year_list = [i for i in range(2005, 2021, 1)]

# Create function that takes in airline data as input, and creates 5 dataframes
# for plotting charts
def compute_dataframes(df):
    # cancellation category count:
    bar_data = df.groupby(['Month', 'CancellationCode'])['Flights'].sum().reset_index()
    # average flight time by airline
    line_data = df.groupby(['Month', 'Reporting_Airline'])['AirTime'].mean().reset_index()
    # diverted airport landings
    div_data = df[df['DivAirportLandings'] != 0]
    # source state count:
    map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()
    # destination state count:
    tree_data = df.groupby(['DestState'])['Flights'].sum().reset_index()
    
    return bar_data, line_data, div_data, map_data, tree_data

# Takes airline data as input to create dataframes of various time delays:
def compute_delays(df):
    avg_carr = df.groupby(['Month', 'Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month', 'Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month', 'Reporting_Airline'])['NASDElay'].mean().reset_index()
    avg_sec = df.groupby(['Month', 'Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month', 'Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    
    return avg_carr, avg_weather, avg_NAS, avg_sec, avg_late

app.layout = html.Div(children = [
                      html.H1('US Domestic Airline Flights Performance',
                              style = {'textAlign': 'center',
                                       'color': '#503D36',
                                       'font-size': 24}),
                      # outer division:
                      html.Div([
                          # first inner division:
                          html.Div([
                              # division for dropdown
                              html.Div([html.H2('Report Type: ',
                                                style = {'margin-right': '2em'})]),
                                        dcc.Dropdown(id = 'input-type',
                                                     options = [{'label': 'Yearly Airline Performance Report',
                                                                 'value': 'OPT1'},
                                                                {'label': 'Yearly Airline Delay Report',
                                                                 'value': 'OPT2'}],
                                                     placeholder = 'Select a report type',
                                                     style = {'width': '80%', 'height': '3px',
                                                              'textAlign': 'center', 'font-size': '20px'})],
                                        style = {'display': 'flex'}),
                          # second inner division:
                          html.Div([
                              # division for adding a dropdown helper text for year selection
                              html.Div([html.H2('Choose Year: ',
                                                style = {'margin-right': '2em'})]),
                                        dcc.Dropdown(id = 'input-year',
                                                     options = [{'label': i, 'value': i} for i in year_list],
                                                     placeholder = 'Select a year',
                                                     style = {'width': '80%', 'padding': '3px',
                                                              'font-size': '20px',
                                                              'text-align-last': 'center'})],
                                        # place them next to each other using:
                                        style = {'display': 'flex'})]),
                        
                      # add computed graphs
                      # we add an empty division and providing an id
                      # that will be updated during our callback
                      html.Div([ ], id = 'plot1'),
                          html.Div([
                              html.Div([ ], id = 'plot2'),
                              html.Div([ ], id = 'plot3'),
                              html.Div([ ], id = 'plot4'),
                              html.Div([ ], id = 'plot5')],
                              style = {'display': 'flex'})])                             

In [ ]:
# the component property will be children as we have created an empty
# division and passed in dcc.Graph after computation
@app.callback([Output(component_id = 'plot1',
                      component_property = 'children'),
               Output(component_id = 'plot2',
                      component_property = 'children'),
               Output(component_id = 'plot3',
                      component_property = 'children'),
               Output(component_id = 'plot4',
                      component_property = 'children'),
               Output(component_id = 'plot5',
                      component_property = 'children')],
              [Input(component_id = 'input-type',
                     component_property = 'value'),
               Input(component_id = 'input-year',
                     component_property = 'value')],
              # holding output state until the user enters all information
              [State('plot1', 'children'), State('plot2', 'children'),
               State('plot3', 'children'), State('plot4', 'children'),
               State('plot4', 'children')])

# add computation to callback function and return graphs
def get_graps(chart, year, children1, c2, c3, c4, c5):
    
    # select data:
    df = airdata[airdata['Year'] == int(year)]
    
    if chart == 'OPT1':
        # compute required info for generating graphs
        bar_data, line_data, div_data, map_data, tree_data = compute_dataframes(df)
        
        # number of flights under various cancellation categories:
        bar_fig = px.bar(bar_data, x = 'Month', y = 'Flights', color = 'CancellationCode',
                         title = 'Monthly Flight Cancellation')
        # average flight time by airline:
        line_fig = px.line(line_data, x = 'Month', y = 'AirTime', color = 'Reporting_Airline',
                           title = 'Monthly Flight Time')
        
        # % of diverted airport landings per airline:
        pie_fig = px.pie(div_data, values = 'Flights', names = 'Reporting_Airline',
                         title = 'Share of Flights by Airline')
        
        # number of flights from each state using choropleth:
        map_fig = px.choropleth(map_data, locations = 'OriginState',
                                color = 'Flights', hover_data = ['OriginState', 'Flights'],
                                locationmode = 'USA-states', # set to plot as US
                                color_continuous_scale = 'GnBu',
                                range_color = [0, map_data['Flights'].max()])
        map_fig.update_layout(title_text = 'Number of Flights from Origin State',
                              geo_scope = 'usa')
        
        # number of flights flying to each state from each airline:
        #tree_fig = px.treemap(names = [], parents = [])
        #tree_fig.update_traces(root_color = 'lightgrey')
        #tree_fig.update_layout(margin = dict(t = , l = , r = , b = )
        tree_fig = px.treemap(tree_data, path = ['Month', 'DestStateName'], values = 'Flights')
        tree_fig.update_layout(margin = dict(t = 50, l = 25, r = 2, b = 25))
        
        return[dcc.Graph(figure = tree_fig),
               dcc.Graph(figure = pie_fig),
               dcc.Graph(figure = map_fig),
               dcc.Graph(figure = bar_fig),
               dcc.Graph(figure = line_fig)]
    
    else:
        # compute required information for creating graph from the data
        avg_carr, avg_weather, avg_NAS, avg_sec, avg_late = compute_delays(df)
        
        carrier_fig = px.line(avg_carr, x = 'Month', y = 'CarrierDelay', 
                              color = 'Reporting_Airline',
                              title = 'Average Carrier Delay (min) by Airline')
        weather_fig = px.line(avg_weather, x = 'Month', y = 'WeatherDelay',
                              color = 'Reporting_Airline',
                              title = 'Average Weather Delay (min) by Airline')
        nas_fig = px.line(avg_NAS, x = 'Month', y = 'NASDelay', 
                          color = 'Reporting_Airline',
                          title = 'Average NAS Delay (min) by Airline')
        sec_fig = px.line(avg_sec, x = 'Month', y = 'SecurityDelay',
                          color = 'Reporting_Airline',
                          title = 'Average Security Delay (min) by Airline')
        late_fig = px.line(avg_late, x = 'Month', y = 'LateAircraftDelay', 
                           color = 'Reporting_Airline',
                           title = 'Average Late Arrival Delay (min) by Airline')
        
        return[dcc.Graph(figure = carrier_fig),
               dcc.Graph(figure = weather_fig),
               dcc.Graph(figure = nas_fig),
               dcc.Graph(figure = sec_fig),
               dcc.Graph(figure = late_fig)]
    
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader = False)
    
# dev_tools_ui=False, dev_tools_props_check=False 

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
